In [1]:
#load dragonn tutorial utilities 
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Ingesting data into tileDB 

In [9]:
from seqdataloader.dbingest import * 

The header of the input task file should contain (one or more) of the following fields: 
    * dataset (this one's required -- it's a unique label for your dataset) 
    * pval_bigwig 
    * fc_bigwig 
    * count_bigwig_plus_5p 
    * count_bigwig_minux_5p
    * idr_peak
    * overlap_peak 
    * ambig_peak 
    
The file paths can be either local or web-based URL's. 

In [7]:
!cat tasks.dbingest.tsv

dataset	idr_peak	fc_bigwig	ambig_peak
ENCFF209DJG	https://www.encodeproject.org/files/ENCFF209DJG/@@download/ENCFF209DJG.bed.gz	https://www.encodeproject.org/files/ENCFF842XRQ/@@download/ENCFF842XRQ.bigWig	hg38.blacklist.bed.gz


You can run the ingest code as a python function: 

In [10]:
args={"tiledb_metadata":"tasks.dbingest.tsv",
      "tiledb_group":"hepg2_dnase_encode",
     "overwrite":True,
     "chrom_sizes":"hg38.chrom21.sizes",
     "chrom_threads":1,
     "task_threads":1,
     "write_threads":1}

ingest(args)

loaded tiledb metadata
loaded chrom sizes
tiledb group already exists
got data dict
parsed pool inputs
made pool!
chrom:chr21
size:46709983
array_out_name:hepg2_dnase_encode/ENCFF209DJG.chr21
data_dict:{'idr_peak':            0         1         2  3  4  5      6  7  8   9
0       chr1    128629    128779  .  0  .   10.0 -1 -1  75
1       chr1    181415    181565  .  0  .  131.0 -1 -1  75
2       chr1    190825    190975  .  0  .   70.0 -1 -1  75
3       chr1    267990    268140  .  0  .  133.0 -1 -1  75
4       chr1    629155    629305  .  0  .  603.0 -1 -1  75
...      ...       ...       ... .. .. ..    ... .. ..  ..
178371  chrY  56847895  56848045  .  0  .   31.0 -1 -1  75
178372  chrY  56848355  56848505  .  0  .   47.0 -1 -1  75
178373  chrY  56848861  56849011  .  0  .   23.0 -1 -1  75
178374  chrY  56849334  56849484  .  0  .   58.0 -1 -1  75
178375  chrY  56851495  56851645  .  0  .   71.0 -1 -1  75

[178376 rows x 10 columns], 'fc_bigwig': 'https://www.encodeproject.org/file

AttributeError: 'ProcessPoolExecutor' object has no attribute 'terminate'

got:fc_bigwig for chrom:chr21
store_summits:False
summit_indicator:None
got:ambig_peak for chrom:chr21
starting to write output
got cur vals
idr_peak
dict_to_write[key].shape:(46709983,)
fc_bigwig
dict_to_write[key].shape:(46709983,)
ambig_peak
dict_to_write[key].shape:(46709983,)
updated data dict for writing
finalizing the write
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
length of pool inputs:48
made pool
start:0, end:1000000
start:1000000, end:2000000
start:2000000, end:3000000
start:3000000, end:4000000
start:4000000, end:5000000
start:5000000, end:6000000
start:6000000, end:7000000
start:7000000, end:8000000
start:8000000, 

Or you can run the code as a script: 

In [ ]:
!db_ingest --tiledb_metadata tasks.dbingest.tsv \
       --tiledb_group hepg2_dnase_encode \
       --overwrite \
       --chrom_sizes hg38.chrom21.sizes \
       --chrom_threads 1 \
       --task_threads 1 \
       --write_threads 1

In [ ]:
#we can examine the array 
import tiledb 
data=tiledb.DenseArray("hepg2_dnase_encode/ENCFF209DJG.chr21",'r')
subset=data[30000000:31000000]
print(subset.keys())

In [ ]:
subset['fc_bigwig'][0:1000]

In [ ]:
subset['idr_peak'][0:1000]

## Genomewide classification labels 

In [ ]:
from seqdataloader.labelgen import *
classification_params={
    'task_list':"tasks.labelgen.tsv",
    'outf':"classificationlabels.SummitWithin200bpCenter.tsv.gz",
    'output_type':'gzip',
    'chrom_sizes':'hg38.chrom.sizes',
    'chroms_to_keep':['chr21'],
    "store_positives_only":True,
    'bin_stride':50,
    'left_flank':400,
    'right_flank':400,
    'bin_size':200,
    'task_threads':10,
    'chrom_threads':4,
    'allow_ambiguous':True,
    'labeling_approach':'peak_summit_in_bin_classification'
    }
genomewide_labels(classification_params)



## Genomewide regression labels 

In [ ]:
regression_params={
    'task_list':"tasks.labelgen.tsv",
    'outf':"regressionlabels.all_genome_bins_regression.hdf5",
    'output_type':'hdf5',
    'chrom_sizes':'hg38.chrom.sizes',
    'store_values_above_thresh': 0,
    'chroms_to_keep':['chr21'],
    'bin_stride':50,
    'left_flank':400,
    'right_flank':400,
    'bin_size':200,
    'threads':10,
    'subthreads':4,
    'labeling_approach':'all_genome_bins_regression'
    }
genomewide_labels(regression_params)


let's examine the output dataframe for the regression case: 

In [ ]:
regression_data=pd.read_hdf("regressionlabels.all_genome_bins_regression.hdf5")

In [ ]:
regression_data.head()

In [ ]:
regression_negatives=pd.read_hdf("universal_negatives.regressionlabels.all_genome_bins_regression.hdf5")
regression_negatives.head

for the classification case, we specified "store_positives_only", so the script generated two dataframes: 
    * Universal negatives 
    * Dataframe where each bin is >0 for at least one task 

In [ ]:
classification_pos=pd.read_csv("classificationlabels.SummitWithin200bpCenter.tsv.gz",sep='\t',header=0)

In [ ]:
classification_pos.head()

In [ ]:
classification_neg=pd.read_csv("universal_negatives.classificationlabels.SummitWithin200bpCenter.tsv.gz",sep='\t',header=0)

In [ ]:
classification_neg.head()